In [2]:
from utils import (create_reference_hap, generate_fragments, 
                   cluster_fragments, simulate_haplotypes_errors,
                   generate_matrix_for_visualization, 
                   get_overlapping_fragments_for_variants_sites,
                   get_probability_fragments_from_same_fragment,
                   remove_false_variants)

from copy import deepcopy

ref_length = int(30)
H12, false_variant_locs = create_reference_hap(ref_length=ref_length, 
                                               false_variance = 0.1)
ref_H = deepcopy(H12)
samples, st_en = generate_fragments(ref_H,
                                    read_length=5, 
                                    std_read_length=1, 
                                    coverage=6,)


hap_samples = deepcopy(samples)
start_end = deepcopy(st_en)
S1, ST_EN = cluster_fragments(hap_samples, st_en)



[11 20 26]
0 0 0 0 0 1 1 0 1 1 0 0 1 0 1 1 0 0 1 1 0 1 1 1 0 0 0 1 0 0
1 1 1 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 1 1




In [3]:
er_hap_samples, sts_ens, frag_file_contents = simulate_haplotypes_errors(hap_samples = S1, 
                                                                         reads_st_en = ST_EN,
                                                                         false_variant_locs = false_variant_locs)

In [4]:
generate_matrix_for_visualization(ref_H, false_variant_locs, # er_hap_samples, sts_ens)
                                  S1, ST_EN)


False variant locations are [11, 20, 26]
reference haplotype values for the variants are:


0 0 0 0 0 1 1 0 1 1 0 0 1 0 1 1 0 0 1 1 0 1 1 1 0 0 0 1 0 0
1 1 1 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 1 1


0 0 0 0 0 - - - - - - - - - - - - - - - - - - - - - - - - -
1 1 1 - - - - - - - - - - - - - - - - - - - - - - - - - - -
- 1 1 1 1 0 0 - - - - - - - - - - - - - - - - - - - - - - -
- - - 0 0 1 1 0 1 - - - - - - - - - - - - - - - - - - - - -
- - - 1 1 0 0 1 - - - - - - - - - - - - - - - - - - - - - -
- - - 1 1 0 - - - - - - - - - - - - - - - - - - - - - - - -
- - - - 1 0 0 1 0 - - - - - - - - - - - - - - - - - - - - -
- - - - 1 0 0 - - - - - - - - - - - - - - - - - - - - - - -
- - - - - - 0 1 0 0 1 1 - - - - - - - - - - - - - - - - - -
- - - - - - - 1 0 0 1 0 - - - - - - - - - - - - - - - - - -
- - - - - - - 1 0 0 1 - - - - - - - - - - - - - - - - - - -
- - - - - - - 1 0 0 1 - - - - - - - - - - - - - - - - - - -
- - - - - - - - 1 1 0 0 1 - - - - - - - - - - - - - - - - -
- - - 

In [5]:
remove_false_variants(S1, ST_EN, 0.02, 0.0003, ref_H)

> c:\users\91821\documents\github\hapcut_preprocessing\utils.py(344)remove_false_variants()
    342     import pdb;pdb.set_trace()
    343     while True:
--> 344         likelihood_false_variants = []
    345         for i in range(len(ref_H[0])):
    346             likelihood_false_variants.append((calculate_likelihood_of_heterozygous_site(reads, st_en, i, qual, ref_H), i))

> c:\users\91821\documents\github\hapcut_preprocessing\utils.py(345)remove_false_variants()
    343     while True:
    344         likelihood_false_variants = []
--> 345         for i in range(len(ref_H[0])):
    346             likelihood_false_variants.append((calculate_likelihood_of_heterozygous_site(reads, st_en, i, qual, ref_H), i))
    347         pdb.set_trace()

> c:\users\91821\documents\github\hapcut_preprocessing\utils.py(348)remove_false_variants()
    346             likelihood_false_variants.append((calculate_likelihood_of_heterozygous_site(reads, st_en, i, qual, ref_H), i))
    347         pdb.se

BdbQuit: 

In [39]:
print(false_variant_locs)
res

[13 22 25]


[(5.324177199394176e-12, 23),
 (9.184997998436638e-11, 19),
 (1.1009455514679497e-08, 20),
 (2.4501060351892216e-08, 17),
 (5.6072650762281196e-08, 21),
 (6.044219720004228e-07, 13),
 (1.4257579108034304e-06, 15),
 (2.9192352046435855e-06, 18),
 (3.2221714671991203e-06, 24),
 (7.043625959125479e-06, 14),
 (1.4867828454595327e-05, 16),
 (0.00025026324495828984, 26),
 (0.0002977469093249775, 22),
 (0.0007716671965203708, 11),
 (0.0012807270859502072, 9),
 (0.0017052756272155814, 6),
 (0.0018880628215111302, 12),
 (0.003926919318680138, 28),
 (0.003967007988244284, 5),
 (0.004092172293601708, 27),
 (0.018898226303321053, 4),
 (0.019677787693010755, 7),
 (0.019693509730169152, 10),
 (0.020095752597280313, 8),
 (0.04355466361783209, 2),
 (0.04355466361783209, 3),
 (0.09243814269135009, 25),
 (0.2044385315539353, 0),
 (0.2044385315539353, 1),
 (nan, 29)]

In [80]:

over_S, over_st_en =  get_overlapping_fragments_for_variants_sites(S1, ST_EN, 22)
generate_matrix_for_visualization(ref_H, false_variant_locs, # er_hap_samples, sts_ens)
                                  over_S, over_st_en)

False variant locations are [3, 5, 6]
reference haplotype values for the variants are:


1 0 0 1 0 1 1 0 0 1 0 1 1 0 0 0 1 1 1 1 1 0 0 1 1 0 0 0 1 0
0 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 0 1 1 1 0 1


- - - - - - - - - - - - - - - - 0 0 0 0 0 1 - - - - - - - -
- - - - - - - - - - - - - - - - - - - 0 0 1 1 - - - - - - -
- - - - - - - - - - - - - - - - - - - 1 1 0 0 - - - - - - -
- - - - - - - - - - - - - - - - - - - 0 0 1 1 0 - - - - - -
- - - - - - - - - - - - - - - - - - - 0 0 1 1 0 0 - - - - -
- - - - - - - - - - - - - - - - - - - 1 1 0 0 1 1 0 - - - -
- - - - - - - - - - - - - - - - - - - - - 1 1 0 - - - - - -
- - - - - - - - - - - - - - - - - - - - - 0 0 1 1 - - - - -
- - - - - - - - - - - - - - - - - - - - - 0 0 1 1 0 0 - - -


In [4]:
print(get_probability_fragments_from_same_fragment(S1[2:2+2], ST_EN[2:2+2], 3, 0.02))
# for i in range(len(S1)-2):
#     print(get_probability_fragments_from_same_fragment(S1[i:i+2], ST_EN[i:i+2], 3, 0.02)

NameError: name 'get_probability_fragments_from_same_fragment' is not defined

Notes: We want to incorporate sequencing errors in the fragments f1 and f2 and get the confidence whether f1 and f2 belong from the same haplotype. Use some probablity to guess whether f1 and f2 belong to the same haplotype. 

Now once we have a probablity that f1 and f2 belong to the same haplotype, we want to check whether a variant is a false variant.

What is a false variant ? 
False variant is a location where the alleles are homozygous but the sequencing 
assumes / wrongly interprets the site as heterozygous. 
Hitherto, in the fragments (for these false variants) there would be a substantial amount of fragments where this site has the same allele say 0 for 
both fragments coming from both H1 and H2 (as it is not really a variant hence same allele is observed irrespective sample from H1 or H2 (0, 0) in the given position).

However for sequencing errors the errors are randomly distributed among different variants for different fragments.

Problem statement, we want to identify these false variant locations for the 
sequenced data, as these become difficult to handle while performing Hap assembly.

When we have a good coverage for a particular false variant, 
if we have fragment f1 which belongs to H1 and a fragment f2 which belongs to H2, the value for the variants are complement of each other except for the false variant. 

Now we dont know whether the fragment belongs to either H1 or H2, 